In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import skimage.io as io

from PIL import Image

import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

import cv2

from pathlib import Path

import sklearn.feature_extraction
import sklearn.feature_extraction
from skimage.segmentation import expand_labels, watershed
import skimage.morphology

from sklearn.cluster import spectral_clustering

from skimage import morphology


import deepdrr
from deepdrr import Volume
from deepdrr import geo
from deepdrr.utils import test_utils

import nibabel as nib

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# unique ops

# crop - done
# split loose - done
# dilate - done
# erode - done
# downscale 2x - done
# upscale 2x - done
# spectral cluster - done
# mode blur - done

In [4]:
# vol_path = Path("/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/pulmonary_vein.nii.gz")
vol_path = Path("/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/skull.nii.gz")
print(vol_path)
print(vol_path.exists())

mesh_path = Path("/home/pelvisvr/PrephixData/ct_meshes/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/skull.stl")
print(mesh_path)
print(mesh_path.exists())

/home/pelvisvr/PrephixData/ct_datasets/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/skull.nii.gz
True
/home/pelvisvr/PrephixData/ct_meshes/PrephixNMDID/nifti/BONE_H-N-UXT_3X3/case-100065/segmentations/skull.stl
True


In [5]:
vol = Volume.from_nifti(vol_path, segmentation=True)
vol

In [6]:
vol.shape

(512, 512, 236)

In [7]:
vol_poly = vol.isosurface(decimation=0, smooth_iter=0, smooth=False,)
vol_poly.save("mesh.stl")
print(vol_poly.bounds)

(-27.361328125, 123.048828125, -247.35638427734375, -2.31732177734375, -642.5, -30.5)


In [8]:
vol_cropped = vol.shrink(padding=5)

In [9]:
vol_cropped.shape

(160, 255, 213)

In [10]:
vol_poly = vol_cropped.isosurface(decimation=0, smooth_iter=0, smooth=False,)
vol_poly.save("cropped_mesh.stl")
print(vol_poly.bounds)


(-27.361328125, 123.048828125, -247.35638427734375, -2.31732177734375, -642.5, -30.5)


In [11]:
bodies = vol_cropped.split_bodies()

In [12]:
vol_data = vol_cropped.data

# vol_data = morphology.binary_erosion(vol_data, footprint=morphology.ball(5))
vol_data = morphology.binary_dilation(vol_data, footprint=morphology.ball(5))

In [13]:
float_labels = vol_data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas2.nii.gz')

In [14]:
body = bodies[0]

body.shape

(160, 255, 213)

In [15]:
body_shrunk = body.shrink(padding=5)
print(body_shrunk.shape)

(160, 216, 79)


In [16]:
shrunk_downscaled = body_shrunk.downscale((3,3,1), segmentation=True)
print(shrunk_downscaled.shape)

float_labels = shrunk_downscaled.data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas3.nii.gz')

(54, 72, 79)


In [17]:
segmented = shrunk_downscaled.cluster_seg(segmentation=True, n_clusters=9, expand=10)

float_labels = segmented.data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas4.nii.gz')

/home/pelvisvr/mambaforge/envs/prephix/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:285: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [18]:
segmented_upscaled = segmented.upscale((3,3,1), segmentation=True, smooth_seg=(3,3,3))

float_labels = segmented_upscaled.data.astype(np.float32)
nii = nib.Nifti1Image(float_labels, np.eye(4))
nib.save(nii, 'asdfas5.nii.gz')

In [19]:
print(segmented_upscaled.shape)
print(body_shrunk.shape)

(162, 216, 79)
(160, 216, 79)


In [20]:
split_segs = segmented_upscaled.split_seg()
print(len(split_segs))

9


In [61]:
part_surfaces = [seg.isosurface(decimation=0, smooth_iter=0, smooth=False,convert_to_LPS=True) for seg in split_segs]

In [62]:
import zipfile
with zipfile.ZipFile("parts.zip", "w") as zf:
    for i, part in enumerate(part_surfaces):
        part.save(f"part_{i}.stl")
        zf.write(f"part_{i}.stl")

In [63]:
# pv load mesh_path
import pyvista as pv
mesh = pv.read(str(mesh_path))

# save to whole.stl
mesh.save("whole.stl")

In [64]:
print(part_surfaces[0].bounds)

(-128.029296875, 33.337890625, 177.62982177734375, 252.33685302734375, -222.5, -18.5)


In [65]:
print(mesh.bounds)

(-122.7006607055664, 26.0391845703125, 4.282985687255859, 246.35723876953125, -640.7942504882812, -30.562864303588867)


In [66]:
result = mesh.boolean_intersection(part_surfaces[0])


In [67]:
result

PolyData (0x7add908c02e0)
  N Cells:    15999
  N Points:   10303
  N Strips:   0
  X Bounds:   -1.201e+02, 1.237e+01
  Y Bounds:   1.820e+02, 2.464e+02
  Z Bounds:   -1.954e+02, -4.323e+01
  N Arrays:   8

In [78]:
# load mesh
import trimesh
mesh = trimesh.load_mesh(mesh_path)

split_meshes = list(Path(".").glob("part_*.stl"))
print(mesh.bounds)

for i, surf in enumerate(split_meshes):
    surf = trimesh.load_mesh(surf)
    # boolean intersection
    print(surf.bounds)
    split = mesh.intersection(surf, engine="manifold")
    # save split mesh
    split.export(f"split_{i}.stl")

import zipfile
with zipfile.ZipFile("split.zip", "w") as zf:
    # glob for split meshes
    split_meshes = Path(".").glob("split_*.stl")
    for split_mesh in split_meshes:
        zf.write(split_mesh)
        print(split_mesh)


[[-122.70066071    4.28298569 -640.79425049]
 [  26.03918457  246.35723877  -30.5628643 ]]
[[ -71.25195312   37.18060303 -189.5       ]
 [  21.38476562  105.91107178  -93.5       ]]
[[ -54.31835938   81.00872803 -225.5       ]
 [  33.33789062  246.36029053  -66.5       ]]
[[ -26.42773438   37.18060303 -255.5       ]
 [  33.33789062  157.70794678 -141.5       ]]
[[-116.07617188   37.18060303 -255.5       ]
 [  -2.52148438  144.75872803 -156.5       ]]
[[-128.02929688   43.15716553 -180.5       ]
 [ -56.31054688  109.89544678  -99.5       ]]
[[-128.02929688  177.62982178 -222.5       ]
 [  33.33789062  252.33685303  -18.5       ]]
[[-128.02929688   87.98138428 -234.5       ]
 [ -29.41601562  241.37982178  -69.5       ]]
[[-128.02929688   37.18060303 -141.5       ]
 [  33.33789062  223.45013428  -18.5       ]]
[[ -19.45507812   52.12200928 -168.5       ]
 [  33.33789062  124.83685303  -84.5       ]]
split_7.stl
split_5.stl
split_4.stl
split_2.stl
split_1.stl
split_6.stl
split_3.stl
split_

In [60]:
print(np.max(surf.vertices, axis=0), np.min(surf.vertices, axis=0))
print(np.max(mesh.vertices, axis=0), np.min(mesh.vertices, axis=0))

[ 33.33789062 124.83685303 -84.5       ] [ -19.45507812   52.12200928 -168.5       ]
[ 26.03918457 246.35723877 -30.5628643 ] [-122.70066071    4.28298569 -640.79425049]


In [79]:
split_parts = body_shrunk.split_by_segmentation(segmented_upscaled.data)
print(len(split_parts))

9


In [80]:
# # make an isosurface for each part
# for i, part in enumerate(split_parts):
#     part_poly = part.isosurface(decimation=0, smooth_iter=0, smooth=False,)
#     part_poly.save(f"part_{i}.stl")

# zip parts
import zipfile
with zipfile.ZipFile("parts.zip", "w") as zf:
    for i, part in enumerate(split_parts):
        part_poly = part.isosurface(decimation_points=20000,smooth=False,convert_to_LPS=True,)
        # part_poly = part.isosurface(decimation=0, smooth_iter=0, smooth=False,)
        part_poly.save(f"part_{i}.stl")
        zf.write(f"part_{i}.stl")

In [ ]:
# shrunk_upscaled = shrunk_downscaled.upscale((4,4,2), segmentation=True)
# print(shrunk_upscaled.shape)